STEP 1 — Install & Import Libraries

In [3]:
# Install required package (run once)
!pip install tensorflow tensorflow-hub matplotlib boto3

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/tensorflow_hub/__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


STEP 2 — Configuration & Paths

In [4]:
print("TensorFlow version:", tf.__version__)

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 300

# Local directory (downloaded from S3 previously)
train_dir = "synthetic_data/Training Data"

print("Training directory:", train_dir)
print("Epochs:", EPOCHS)


TensorFlow version: 2.16.2
Training directory: synthetic_data/Training Data
Epochs: 300


STEP 3 — Load Data (20% Validation Split, NO Augmentation)

In [5]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training",
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation",
    shuffle=False
)

print("Class indices:", train_generator.class_indices)


Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Class indices: {'Loose': 0, 'Tight': 1}


STEP 4 — Load EfficientNet-Lite Backbone (Frozen)

In [6]:
# TF-Hub EfficientNet-Lite0 (ImageNet pretrained)
EFFICIENTNET_LITE_URL = (
    "https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2"
)

# Load feature extractor
base_model = hub.KerasLayer(
    EFFICIENTNET_LITE_URL,
    input_shape=(224, 224, 3),
    trainable=False   # IMPORTANT: frozen backbone
)

print("EfficientNet-Lite backbone loaded and frozen")


2025-12-23 13:39:49.526958: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-23 13:39:49.573408: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-23 13:39:49.574502: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

EfficientNet-Lite backbone loaded and frozen


STEP 5 — Build Classification Head

In [8]:
from tensorflow.keras.layers import Input

# Create input tensor
inputs = Input(shape=(224, 224, 3))

# Pass inputs through EfficientNet-Lite backbone
x = base_model(inputs)   # ✅ THIS is the key fix

# Add classification head
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)

# Create final model
model = Model(inputs=inputs, outputs=outputs)

# Print model summary
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 keras_layer (KerasLayer)    (None, 1280)              3413024   
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 3577121 (13.65 MB)
Trainable params: 164097 (641.00 KB)
Non-trainable params: 3413024 (13.02 MB)
_________________________________________________________________


STEP 6 — Compile Model

In [9]:
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=[BinaryAccuracy(name="accuracy", threshold=0.5)]
)

print("Model compiled successfully")


Model compiled successfully


STEP 7 — Callbacks

In [10]:
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=20,
    restore_best_weights=True,
    verbose=1
)

lr_scheduler = ReduceLROnPlateau(
    monitor="loss",
    factor=0.1,
    patience=5,
    min_lr=1e-7,
    verbose=1
)


STEP 8 — Train Model

In [11]:
print("Training started...")

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stopping, lr_scheduler],
    verbose=1
)

print("Training completed")


Training started...
Epoch 1/300


2025-12-23 13:42:12.718142: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2025-12-23 13:42:13.012793: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 406.01MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-12-23 13:42:13.659958: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 908.41MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-12-23 13:42:13.687766: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 940.06MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there co

 1/50 [..............................] - ETA: 12:56 - loss: 0.7194 - accuracy: 0.5312

2025-12-23 13:42:15.189541: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 406.01MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


 2/50 [>.............................] - ETA: 22s - loss: 0.7879 - accuracy: 0.4531  

2025-12-23 13:42:15.504639: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 908.41MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-12-23 13:42:15.532261: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 940.06MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


50/50 [==============================] - 23s 149ms/step - loss: 0.4807 - accuracy: 0.7750 - val_loss: 0.2440 - val_accuracy: 0.9625 - lr: 1.0000e-04
Epoch 2/300
50/50 [==============================] - 6s 109ms/step - loss: 0.2094 - accuracy: 0.9481 - val_loss: 0.1296 - val_accuracy: 0.9825 - lr: 1.0000e-04
Epoch 3/300
50/50 [==============================] - 6s 112ms/step - loss: 0.1314 - accuracy: 0.9688 - val_loss: 0.0898 - val_accuracy: 0.9875 - lr: 1.0000e-04
Epoch 4/300
50/50 [==============================] - 6s 112ms/step - loss: 0.0923 - accuracy: 0.9819 - val_loss: 0.0660 - val_accuracy: 0.9925 - lr: 1.0000e-04
Epoch 5/300
50/50 [==============================] - 6s 111ms/step - loss: 0.0737 - accuracy: 0.9844 - val_loss: 0.0523 - val_accuracy: 0.9950 - lr: 1.0000e-04
Epoch 6/300
50/50 [==============================] - 6s 110ms/step - loss: 0.0615 - accuracy: 0.9894 - val_loss: 0.0426 - val_accuracy: 0.9975 - lr: 1.0000e-04
Epoch 7/300
50/50 [==============================] 

STEP 9 — Save Model

In [12]:
model.save("clothe_classifier_efficientnet_lite.h5")
print("Model saved as clothe_classifier_efficientnet_lite.h5")


Model saved as clothe_classifier_efficientnet_lite.h5


/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
